In [1]:
# !git clone https://github.com/facebookresearch/av_hubert.git

# %cd av_hubert
# !git submodule init
# !git submodule update
# !pip install scipy
# !pip install sentencepiece
# !pip install python_speech_features
# !pip install scikit-video

# %cd fairseq
# !pip install ./

In [2]:
# import cv2
# net = cv2.dnn.readNetFromONNX('../../avhencoder.onnx')

In [4]:
# import numpy as np
# sample = {'audio': np.random.randn(1, 104, 500),
#                           'video': np.random.randn(1, 1, 500, 112, 112)}
# net.forward([np.random.randn(1,104,500), np.random.randn(1,1,500,112,112)])

In [4]:
!wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O finetune-model.pt

--2022-07-17 10:57:17--  https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1928063847 (1.8G) [binary/octet-stream]
Saving to: ‘finetune-model.pt’

finetune-model.pt     0%[                    ]  13.64M  4.60MB/s               ^C


In [6]:
%cd av_hubert/avhubert
import cv2
import tempfile
from argparse import Namespace
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.configs import GenerationConfig
from IPython.display import HTML
import torch.onnx
import time

video_path, ckpt_path = "../roi.mp4", "../finetune-model.pt"
user_dir = "./"
num_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))
data_dir = tempfile.mkdtemp()
tsv_cont = ["/\n", f"test-0\t{video_path}\t{None}\t{num_frames}\t{int(16_000*num_frames/25)}\n"]
label_cont = ["DUMMY\n"]
with open(f"{data_dir}/test.tsv", "w") as fo:
    fo.write("".join(tsv_cont))
with open(f"{data_dir}/test.wrd", "w") as fo:
    fo.write("".join(label_cont))
utils.import_user_module(Namespace(user_dir=user_dir))
modalities = ["video", "audio"]
gen_subset = "test"
gen_cfg = GenerationConfig(beam=20)
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
models = [model.eval() for model in models]
# saved_cfg.task.modalities = modalities
# saved_cfg.task.data = data_dir
# saved_cfg.task.label_dir = data_dir
# task = tasks.setup_task(saved_cfg.task)
# task.load_dataset(gen_subset, task_cfg=saved_cfg.task)
# generator = task.build_generator(models, gen_cfg)

# # print(generator)

# def decode_fn(x):
#     dictionary = task.target_dictionary
#     symbols_ignore = generator.symbols_to_strip_from_output
#     symbols_ignore.add(dictionary.pad())
#     return task.datasets[gen_subset].label_processors[0].decode(x, symbols_ignore)

# itr = task.get_batch_iterator(dataset=task.dataset(gen_subset)).next_epoch_itr(shuffle=False)
# sample = next(itr)
# sample = utils.move_to_cuda(sample)
# hypos = task.inference_step(generator, models, sample)
# ref = decode_fn(sample['target'][0].int().cpu())
# hypo = hypos[0][0]['tokens'].int().cpu()
# hypo = decode_fn(hypo)

[Errno 2] No such file or directory: 'av_hubert/avhubert'
/mnt/c/Codes/GSoC/GSoC22/Github_updates/av_hubert/avhubert


OSError: Model file not found: ../finetune-model.pt

In [2]:
# models[0].encoder
source ={'audio': torch.randn(1, 104, 500),
                          'video': torch.randn(1, 1, 500, 112, 112)}
models[0].encoder.w2v_model.prepare_for_onnx_export_()
# Had to do a lot of hardcoding to make it work
torch.onnx.export(models[0].encoder.w2v_model.cpu(), (source, {"mask":False, 'features_only':True}), '../encoder.onnx', opset_version=15, input_names=['audio', 'video'], output_names=['output'])

/mnt/c/Codes/GSoC/GSoC22/Github_updates/av_hubert/avhubert/hubert.py:602: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if mask and self.masking_type == 'input':
/home/shiv/.local/lib/python3.8/site-packages/fairseq/modules/multihead_attention.py:151: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert embed_dim == self.embed_dim
/home/shiv/.local/lib/python3.8/site-packages/fairseq/modules/multihead_attention.py:152: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python v

In [ ]:
# try to load model and run it directly without importing anything. If ther's some ssue then changing the sequencegenrator will work otherwise not. We'll have to convert it using some other mechnanism.